In [17]:
from transformers import Trainer, TrainingArguments, DistilBertForSequenceClassification, DistilBertTokenizer
from datasets import Dataset, DatasetDict
from sklearn.preprocessing import LabelEncoder
import joblib
import pandas as pd
from transformers import DataCollatorWithPadding


# Cargar el archivo CSV
def cargar_csv(ruta_archivo):
    df = pd.read_csv('dataset.csv',sep=';')
    # Verificar que las columnas necesarias existan
    if 'text' not in df.columns or 'label' not in df.columns:
        raise ValueError("El archivo CSV debe contener las columnas 'text' y 'label'.")
    return df

# Codificar etiquetas
def codificar_etiquetas(df):
    le = LabelEncoder()
    df['label'] = le.fit_transform(df['label'])
    # Guardar el encoder para usarlo en la predicción
    joblib.dump(le, "modelo_transformers/label_encoder.pkl")
    return df, le

# Tokenizar los datos
def tokenizar(df, tokenizer):
    def tokenize_fn(example):
        return tokenizer(example["text"], truncation=True, padding=True)

    dataset = Dataset.from_pandas(df)
    return dataset.map(tokenize_fn)

# Entrenar el modelo
def entrenar_modelo(dataset, tokenizer):
    # Dividir en conjunto de entrenamiento y validación (80% - 20%)
    train_test = dataset.train_test_split(test_size=0.2)  # Dividir en 80% entrenamiento y 20% validación

    # Modelo de DistilBERT
    model = DistilBertForSequenceClassification.from_pretrained(
        "distilbert-base-multilingual-cased", num_labels=len(set(dataset['label']))
    )

    training_args = TrainingArguments(
        output_dir="modelo_transformers",
        per_device_train_batch_size=4,
        num_train_epochs=3,
        # evaluation_strategy="epoch",   # 👈 evaluar al final de cada época
        # save_strategy="epoch",         # 👈 guardar al final de cada época
        # load_best_model_at_end=True,   # 👈 solo tiene sentido si evaluamos
        logging_dir="logs",
        logging_steps=1,
        report_to="none"
    )
    # Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_test['train'],
        eval_dataset=train_test['test'],
        tokenizer=tokenizer,
        data_collator=DataCollatorWithPadding(tokenizer)
    )

    # Entrenamiento
    trainer.train()

    # Guardar el modelo y tokenizer entrenados
    model.save_pretrained("modelo_transformers")
    tokenizer.save_pretrained("modelo_transformers")


# Cargar el tokenizador de DistilBERT
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-multilingual-cased")

# Ruta del archivo CSV
ruta_archivo = "dataset.csv"  # Cambia la ruta al archivo real

# Cargar y procesar los datos
df = cargar_csv(ruta_archivo)
df, le = codificar_etiquetas(df)

# Tokenizar los datos
dataset = tokenizar(df, tokenizer)

# Entrenar el modelo
entrenar_modelo(dataset, tokenizer)


Map:   0%|          | 0/1900 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-17-658d0b744fa4>:55: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
1,1.438100
2,1.393400
3,1.396400
4,1.345600
5,1.346700
6,1.263200
7,1.297700
8,1.285300
9,1.384300
10,1.446800


In [18]:
import shutil

shutil.make_archive("modelo_transformers", 'zip', "modelo_transformers")


'/content/modelo_transformers.zip'

In [20]:
from google.colab import files

files.download("modelo_transformers.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>